In [2]:
import pandas as pd
import ast as ast
import numpy as np
from typing import *

from task_2_helpers import *
from collections import Counter

from task2_business_KNN import classify_Kmeans


### Data Read

In [12]:
# df_business = pd.read_parquet('data/ATML2024_businesses.parquet')
df_business = pd.read_csv('data/ATML2024_businesses.csv', header=0)

# df_Train_reviews = pd.read_parquet('data/ATML2024_reviews_train.parquet')
df_Train_reviews = pd.read_csv('data/ATML2024_reviews_train.csv', header=0)

# # df_users = pd.read_parquet('data/ATML2024_users.parquet')
# df_users = pd.read_csv('data/ATML2024_users.csv', header=0, nrows=100)

In [4]:
df_business.postal_code.isna().value_counts()

postal_code
False    138145
True         65
Name: count, dtype: int64

In [5]:
zipCodes = dict(df_business.postal_code.value_counts())

val_count = 0
for z in zipCodes.keys():
    try:
        zcode = int(z)
        if zcode >= 19100 and  zcode < 19200:
            val_count += zcode
    except:
        continue

print(val_count)

1128758


### Cleanup business data columns

In [6]:
bussiness_city = ['Phila','Philadelphia', 'Philadelphia', 'Philadelphia (Northeast Philly)', 'Philadelphia PA', 'Philadelphila', 'Philadephia', 'Philiadelphia', 
                  'Philiidelphia', 'Philly', 'Southwest Philadelphia', 'West Philadelphia', 'philadelphia', 'PHILA', 'PHILADELPHIA']

In [8]:
# Convert dictionary columns to the correct types @@@@ Run only once
df_business_ph = df_business[df_business['city'].isin(bussiness_city)].copy()
df_business_ph['attributes'] = df_business_ph['attributes'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) or not pd.isnull(x)  else dict())
df_business_ph['hours'] = df_business_ph['hours'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) or not pd.isnull(x)  else dict())

# df_business['weekend'] = df_business['hours'].apply(lambda x: extract_weekend_value(x))
df_business_ph['hours_list'] = df_business_ph['hours'].apply(lambda x: extract_hours_value(x))


In [9]:
df_business_ph['attributes_list'] = df_business_ph['attributes'].apply(lambda x: sorted(extract_attributes_value(x)))
df_business_ph['categories_list'] = df_business_ph['categories'].apply(lambda x: sorted(map(lambda a: a.strip().lower(), x.split(',')) ) if isinstance(x, str) else [x])

In [11]:
df_business[df_business['city'].isin(bussiness_city)]

,business_id,name,address,city,state,postal_code,latitude,longitude,attributes,categories,hours
2,wm9eoqjytVbC7dQcM4WSTM,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
13,AeNmR681lBdMyqu8eqTTUM,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-..."
17,gCT6F7gqR50heBwQJcaeOR,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,"{'NoiseLevel': ""u'quiet'"", 'GoodForMeal': ""{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20..."
26,g7r5IhhJGdpyFyoWB27NdQ,Bar One,767 S 9th St,Philadelphia,PA,19147,39.939825,-75.157447,"{'Smoking': ""u'no'"", 'NoiseLevel': ""u'average'...","Cocktail Bars, Bars, Italian, Nightlife, Resta...","{'Monday': '16:0-0:0', 'Tuesday': '16:0-0:0', ..."
29,Q_dELKJOieBlN9M42zobjM,DeSandro on Main,4105 Main St,Philadelphia,PA,19127,40.022466,-75.218314,"{'RestaurantsReservations': 'False', 'Caters':...","Pizza, Restaurants, Salad, Soup","{'Tuesday': '17:0-21:30', 'Wednesday': '17:0-1..."
...,...,...,...,...,...,...,...,...,...,...,...
138187,g7UYng0WfLfw5qbFbAaAPw,Intermix,1718 Walnut St,Philadelphia,PA,19103,39.950060,-75.170129,"{'BusinessParking': ""{'garage': False, 'street...","Fashion, Shopping, Women's Clothing","{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'..."
138192,wZZe6z-W5_cBemECik7MDJ,2601 Parkway Apartments,2601 Pennsylvania Ave,Philadelphia,PA,19130,39.968806,-75.180608,{'BusinessAcceptsCreditCards': 'True'},"Real Estate, Apartments, Home Services",NaN
138195,gz-cNmRZWoL459epcgI1U9,Hand & Stone Massage And Facial Spa,"1100 S Columbus Blvd, Ste 24",Philadelphia,PA,19147,39.932756,-75.144504,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Day Spas, Beauty & Spas, Skin Care, Massage","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
138198,QaNNUNpPIvqk6LH1QjG4JL,Shanti Yoga and Ayurveda,"1638 Pine St, Fl 1",Philadelphia,PA,19103,39.945966,-75.169666,"{'ByAppointmentOnly': 'True', 'GoodForKids': '...","Health & Medical, Yoga, Shopping, Naturopathic...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."


### Join Business Data With Ratings Data and Conduct Exploratory Data Analysis (EDA)

In [51]:
df_business_data = df_business_ph[['business_id', 'postal_code', 'attributes_list', 'hours_list', 'categories_list']]
df_ratings_data = df_Train_reviews[['business_id', 'rating', 'useful', 'funny', 'cool', 'text']]

df_data = pd.merge(df_business_data, df_ratings_data, left_on= 'business_id', right_on='business_id')
df_data.head()

,business_id,postal_code,attributes_list,hours_list,categories_list,rating,useful,funny,cool,text
0,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,1,0,2,So I don't know how the other review was doubl...
1,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,0,I love this place.. I love their chunky fried...
2,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,1,I ordered so many cake from here for our famil...
3,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,0,0,0,Ordered a birthday cake here for the first tim...
4,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,1,0,0,This is my favorite bakery in Chinatown! It's ...


In [52]:
# df_data.to_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet')
# print(df_data.shape)

(145048, 10)


### Explanatory Data Analysis

In [ ]:
df_data = pd.read_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet', engine='pyarrow')
print(df_data.shape)
print(df_data.columns)

df_data_res = df_data[df_data['categories_list'].apply(lambda x: any('restaurants' in str(a).lower() for a in x)
                       or any('restaurant' in str(a).lower() for a in x))]
df_data_res.shape


(145048, 10)
Index(['business_id', 'postal_code', 'attributes_list', 'hours_list',
       'categories_list', 'rating', 'useful', 'funny', 'cool', 'text'],
      dtype='object')


In [3]:
# all_uniq_cat = set(np.concatenate(df_data.categories_list.values).ravel()) #df_data.categories_list.values.flatten()
# all_uniq_attr = set(np.concatenate(df_data.attributes_list.values).ravel()) #df_data.categories_list.values.flatten()

#### 2.0 Find out which one of the categories are related to restaurant services

Before deal with the reviews and treds in them, we need to identify which one of the businesses are actually in the restaurant and related area.

In [4]:
business_cat = np.concatenate(df_data.categories_list.values).ravel()
c = Counter(business_cat)

print(f'Total number of distinct categories of business: {len(c)}')

print(f'The most frequest categories : {c.most_common(10)}')

Total number of distinct categories of business: 1006
The most frequest categories : [('restaurants', 103030), ('food', 40393), ('nightlife', 35392), ('bars', 33605), ('american (new)', 23182), ('breakfast & brunch', 19437), ('american (traditional)', 17285), ('sandwiches', 16678), ('coffee & tea', 12653), ('italian', 12218)]


### 2.2 What is the food trend of Philadelphia

In [59]:
df_data = pd.read_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet', engine='pyarrow')
print(df_data.shape)
print(df_data.columns)

df_data_res = df_data[df_data['categories_list'].apply(lambda x: any('restaurants' in str(a).lower() for a in x)
                       or any('restaurant' in str(a).lower() for a in x))]
                                                                     

(145048, 10)
Index(['business_id', 'postal_code', 'attributes_list', 'hours_list',
       'categories_list', 'rating', 'useful', 'funny', 'cool', 'text'],
      dtype='object')


In [60]:
df_data_res.head()

,business_id,postal_code,attributes_list,hours_list,categories_list,rating,useful,funny,cool,text
0,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,1,0,2,So I don't know how the other review was doubl...
1,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,0,I love this place.. I love their chunky fried...
2,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,1,I ordered so many cake from here for our famil...
3,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,0,0,0,Ordered a birthday cake here for the first tim...
4,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,1,0,0,This is my favorite bakery in Chinatown! It's ...


In [61]:
print(df_data_res.shape)

(103033, 10)


In [21]:
# rating_1_cat = set(np.concatenate(df_data[df_data.rating == 1].categories_list.values).ravel())
# rating_2_cat = set(np.concatenate(df_data[df_data.rating == 2].categories_list.values).ravel())
# rating_3_cat = set(np.concatenate(df_data[df_data.rating == 3].categories_list.values).ravel())
# rating_4_cat = set(np.concatenate(df_data[df_data.rating == 4].categories_list.values).ravel())
# rating_5_cat = set(np.concatenate(df_data[df_data.rating == 5].categories_list.values).ravel())